In [1]:
import pandas as pd
from datetime import datetime
import os, sys
import pyautogui
import time
import mysql.connector
import pyperclip
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from env.private_keys.desen import desencriptar
import tkinter as tk
import customtkinter as ctk
import math
import dotenv as env
import psutil
import traceback
import win32gui
import win32process
import psutil


In [2]:
Condicional_NDS = 1

In [3]:

tiempoEntrePasos = 1.5
grupoWSP = "Lideres - GTR (customer)"
publicar = 0
dbTableName = "sadasd"
contactoError = "2604053233"

In [4]:

def iniciarconexion(host,database,username,password):
    cnx = mysql.connector.connect(host=host, database=database, user=username, password=password)
    return cnx


Procesamiento de Nomina

In [5]:
credentials = desencriptar()
cnx = iniciarconexion(credentials['DB_HOST'], 
                      credentials['DB_DATABASE_OMNIA'],
                      credentials['DB_USER'],
                      credentials['DB_PASSWORD'])
cursor = cnx.cursor()
cursor.execute("SELECT usuario_avaya, nombres, apellidos FROM asesores WHERE campana = 'ATT' and puesto = 'REPRESENTANTE' and fecha_baja IS NULL")
nomina = cursor.fetchall()
cursor.close()
cnx.close()
df_nomina = pd.DataFrame(nomina, columns=["ID_Avaya", "nombres","apellidos"])


✅Credenciales desencriptadas correctamente


In [6]:
df_nomina['Nombre'] = df_nomina['apellidos'] + ', ' + df_nomina['nombres']
df_nomina['Nombre'] = df_nomina['Nombre'].str.title()
df_nomina['ID_Avaya'] = df_nomina['ID_Avaya'].astype(int)
df_nomina = df_nomina.drop(axis=1, columns=["nombres", "apellidos"])
df_nomina.head()


,ID_Avaya,Nombre
0,562833,"Caceres Pereira, Andres Federico"
1,562765,"Rodriguez Alfonzo, Naiyely Hecnay"
2,562693,"Pereyra, Maria Norma"
3,562852,"Gutierrez, Mariana Alejandra"
4,562646,"Rubio, Jesica Belen"


In [7]:
#Crear dataframe vacio para almacenar datos de conexion
columnsDfConexion = ["ID", "inicioConexion", "tiempoTranscurrido", "proximoRefrigerio", "Skill"]
df_conexiones = pd.DataFrame(columns=columnsDfConexion)

# Crear dataframe vacio para almacenar datos de plana
columnas = ['Nombre', 'ID', 'Extension', 'Motivo_AUX', 'Estado', 'Tiempo', 'VDN', "Skill", "Fecha", "Hora"]
df_tiempos = pd.DataFrame(columns=columnas)




def iniciar_temporizador(tiempo_inicial):
    # Configurar el modo oscuro
    ctk.set_appearance_mode("dark")
    
    # Crear la ventana principal
    root = ctk.CTk()
    root.title("Temporizador de Cuenta Regresiva")

    # Configurar el tamaño de la ventana
    window_width = 800
    window_height = 300

    # Obtener el tamaño de la pantalla
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # Calcular la posición para centrar la ventana
    position_top = int(screen_height/2 - window_height/2)
    position_right = int(screen_width/2 - window_width/2)

    # Configurar la geometría de la ventana para centrarla
    root.geometry(f"{window_width}x{window_height}+{position_right}+{position_top}")

    # Hacer que la ventana siempre esté en primer plano
    root.attributes("-topmost", True)
    
    # Crear una etiqueta para el mensaje
    mensaje = ctk.CTkLabel(root, text="La consulta se realizará en:", font=("Helvetica", 48))
    mensaje.pack(pady=20)

    # Crear una etiqueta para mostrar el tiempo restante
    label = ctk.CTkLabel(root, font=("Helvetica", 48))
    label.pack(expand=True)

    def cuenta_regresiva(t):
        if t >= 0:
            label.configure(text=str(t))
            root.update()
            time.sleep(1)
            cuenta_regresiva(t-1)
        else:
            label.configure(text="¡Comenzamos!")
            root.update()
            time.sleep(1)
            root.destroy()  # Cierra la ventana

    # Iniciar la cuenta regresiva desde el tiempo inicial proporcionado
    cuenta_regresiva(tiempo_inicial)

    # Ejecutar el bucle principal de tkinter
    root.mainloop()



def interfaceCuentaRegresiva(tiempo_inicial_minutos):
    # Configurar el modo oscuro
    ctk.set_appearance_mode("dark")
    
    # Crear la ventana principal
    root = ctk.CTk()
    root.title("Temporizador de Cuenta Regresiva")

    # Configurar el tamaño de la ventana
    window_width = 800
    window_height = 300

    # Obtener el tamaño de la pantalla
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # Calcular la posición para centrar la ventana
    position_top = int(screen_height/2 - window_height/2)
    position_right = int(screen_width/2 - window_width/2)

    # Configurar la geometría de la ventana para centrarla
    root.geometry(f"{window_width}x{window_height}+{position_right}+{position_top}")

    # Hacer que la ventana siempre esté en primer plano
    root.attributes("-topmost", True)
    
    # Crear una etiqueta para el mensaje
    mensaje = ctk.CTkLabel(root, text="La consulta se realizará en:", font=("Helvetica", 48))
    mensaje.pack(pady=20)

    # Crear una etiqueta para mostrar el tiempo restante en formato mm:ss
    label = ctk.CTkLabel(root, font=("Helvetica", 48))
    label.pack(expand=True)

    def cuenta_regresiva(t):
        if t >= 0:
            mins, secs = divmod(t, 60)
            timeformat = '{:02d}:{:02d}'.format(mins, secs)
            label.configure(text=timeformat)
            root.update()
            time.sleep(1)
            cuenta_regresiva(t-1)
        else:
            label.configure(text="¡Comenzamos!")
            root.update()
            time.sleep(1)
            root.destroy()  # Cierra la ventana

    # Convertir minutos a segundos
    tiempo_inicial_segundos = tiempo_inicial_minutos * 60

    # Iniciar la cuenta regresiva desde el tiempo inicial proporcionado
    cuenta_regresiva(tiempo_inicial_segundos)

    # Ejecutar el bucle principal de tkinter
    root.mainloop()



def click_on_image_when_visible(image_path):
    while True:
        locations = list(pyautogui.locateAllOnScreen(image_path))  # Convertir el generador en una lista

        if locations:  # Verifica si se encontraron ubicaciones
            for location in locations:  # Itera sobre todas las ubicaciones encontradas
                center = pyautogui.center(location)
                pyautogui.click(center)
                return True
            break  # Se sale del bucle una vez que se encuentra y hace clic en la imagen





def ejecutarReporte(reporte):
    
    #Obtener el directorio de trabajo actual
    directorio = os.getcwd()

    #Ruta completa al ejecutable
    path = os.path.join(directorio, reporte)

    #Ejecutar el reporte
    os.startfile(path)

    #Se realiza un click en la imagen del reporte
    click_on_image_when_visible("reporte.PNG")
    pyautogui.moveTo(500,500)



def consultarConcatenarTiempoConexion():

    global df_conexiones  # Declarar la variable como global para poder accederla

    # Crear el DataFrame a partir del portapapeles
    columns = ["Identif. de conexión", "Hora de acceso"]
    df = pd.read_clipboard(usecols=columns)

    # Renombrar las columnas
    df = df.rename(columns={"Hora de acceso": "inicioConexion", "Identif. de conexión": "ID"})

    # Convertir la columna 'inicioConexion' a formato de tiempo
    df['inicioConexion'] = pd.to_datetime(df['inicioConexion'], format='%H:%M').dt.time

    # Ordenar el DataFrame por 'ID' y 'inicioConexion'
    df = df.sort_values(by=['ID', 'inicioConexion'])

    # Eliminar duplicados en 'ID' manteniendo el valor más pequeño de 'inicioConexion'
    df = df.drop_duplicates(subset='ID', keep='first')

    # Función para calcular el tiempo transcurrido en minutos
    def calcular_tiempo_transcurrido(hora_inicio):
        now = datetime.now().time()
        inicio = datetime.combine(datetime.today(), hora_inicio)
        actual = datetime.combine(datetime.today(), now)
        diferencia = actual - inicio
        return diferencia.total_seconds() / 60

    # Agregar la columna 'tiempoTranscurrido'
    df['tiempoTranscurrido_min'] = df['inicioConexion'].apply(calcular_tiempo_transcurrido)

    # Agregar la columna 'proximoRefrigerio'
    df['proximoRefrigerio_min'] = df['tiempoTranscurrido_min'].apply(lambda x: 600 if x < 180 else 1200)

    # Ordenar el DataFrame por 'inicioConexion' de forma descendente
    df = df.sort_values(by='inicioConexion', ascending=False)

    df_conexiones = pd.concat([df_conexiones, df], ignore_index=True)



def navegarCalcularConcatenarReporteConexiones():
    
    global fecha
    pyautogui.write("860;861")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("tab")
    time.sleep(tiempoEntrePasos) 
    pyautogui.write(fecha)
    time.sleep(0.3)
    pyautogui.press("enter")
    time.sleep(3)
    pyautogui.press("alt")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("right")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("down")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("down")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    consultarConcatenarTiempoConexion()
    time.sleep(4)

    #Cerrar reporte
    pyautogui.hotkey("alt","f4")


def tratamientoPlanaTiempoReal(skill):

    global df_tiempos


    df = pd.read_clipboard()
    
    nuevos_nombres = {  'Nombre del agente': 'Nombre', 
                    'Identif. de conexión': 'ID',
                    'Extn': 'Extension',
                    'Motivo AUX': 'Motivo_AUX',
                    'Nombre de VDN': 'VDN'
                    }
    

    df = df.rename(columns=nuevos_nombres)


    # Colocar el valor "HOLD" en Motivo_AUX cuando el valor en Estado sea "OTHER"
    df.loc[df['Estado'] == 'OTHER', 'Motivo_AUX'] = 'HOLD'

    #Dropear columnas y agregar Skill
    df = df.drop('Nivel', axis=1)
    df = df.drop('Split/Skill', axis=1)
    df.drop("Nombre", axis=1 , inplace= True)
    df['Skill'] = skill

    #Asigno el nombre de la persona cruzando df con df_nomina
    df = df.merge(df_nomina, left_on="ID", right_on="ID_Avaya", how= "left")
    df.drop("ID_Avaya", axis=1 , inplace= True)
    # Agregar una nueva columna con la fecha y hora actual en todas las filas
    df['Fecha'] = datetime.now().strftime("%Y-%m-%d")
    df['Hora'] = datetime.now().strftime("%H:%M")

    df_tiempos = pd.concat([df_tiempos, df], ignore_index=True)
    

def navegarCalcularConcatenarPlanaTiempoReal():
    pyautogui.write("860")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(2)
    pyautogui.press("alt")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("right")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("down")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("down")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    tratamientoPlanaTiempoReal(860)
    time.sleep(tiempoEntrePasos) 
    #Reiniciar reporte
    pyautogui.press("alt")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.write("861")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(2)
    pyautogui.press("alt")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("right")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("down")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("down")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos) 
    pyautogui.press("enter")
    tratamientoPlanaTiempoReal(861)
    time.sleep(tiempoEntrePasos) 
    #Cerrar reporte
    pyautogui.hotkey("alt","f4")



def get_intervalo(hora):
    # Convertimos la hora a minutos totales
    total_minutes = hora.hour * 60 + hora.minute
    # Redondeamos hacia abajo al intervalo más cercano de 30 minutos860;861 27/08/24


    interval_start = (total_minutes // 30) * 30
    # Convertimos los minutos de vuelta a la hora y minuto
    interval_hour = interval_start // 60
    interval_minute = interval_start % 60
    # Formateamos el intervalo en formato HH:MM
    return f'{interval_hour:02}:{interval_minute:02}'

In [8]:
def scrapeoNDS (skill):
    # Scrapeo de la informacion
    ejecutarReporte("Plasma GTR - MTZ.acsup")
    time.sleep(tiempoEntrePasos)
    pyautogui.write(skill)
    pyautogui.press("tab")
    pyautogui.press("enter")
    time.sleep(3)
    pyautogui.press("alt")
    time.sleep(tiempoEntrePasos)
    pyautogui.press("right")
    time.sleep(tiempoEntrePasos)
    for x in range(3):
        pyautogui.press("down")   
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos)
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos)
    pyautogui.press("enter")
    time.sleep(tiempoEntrePasos)
    pyautogui.hotkey("alt", "f4")
    time.sleep(tiempoEntrePasos)
    
    # pegamos la tabla en un df
    df = pd.read_clipboard()

    return df
  

In [9]:
# Obtenemos la fecha de hoy
fecha_dim = datetime.now()
intervalo_hora_dim = fecha_dim.hour
intervalo_min_dim = fecha_dim.minute
fecha_dim = fecha_dim.strftime("%Y-%m-%d")


# Condicionamos los intervalos para que nos de un numero cerrado
print(intervalo_hora_dim)
print(intervalo_min_dim)
if intervalo_hora_dim == 8 or intervalo_hora_dim == 9:
    intervalo_hora_dim = f"0{intervalo_hora_dim}"
if intervalo_min_dim >= 0 and intervalo_min_dim < 30:
    intervalo_min_dim = '00'
else:
    intervalo_min_dim = 30

intervalo_dim = f"{intervalo_hora_dim}:{intervalo_min_dim}"
print("fecha", fecha_dim, "Intervalo: ",intervalo_dim)


8
1
fecha 2025-04-17 Intervalo:  08:00


In [10]:
# Consulta dimensionamiento_ppp
def consulta_dimensionamiento_ppp(fecha, intervalo, skill):
    # Conexion db
    cnx = iniciarconexion(credentials['DB_HOST'], 
                      credentials['DB_DATABASE_ATP'],
                      credentials['DB_USER'],
                      credentials['DB_PASSWORD'])    
    cursor = cnx.cursor()
    query = f"""SELECT 
    dp.Fecha,
    dp.Intervalo,
    dp.Skill,
    dp.Volumen as Pronostico,
    dp.TMO as TMO_prono,
    dp.Improductividad as Improductividad_prono,
    dp.Ausentismo as Ausentismo_prono,
    dp.Raw_Positions as Erlang,
    da.RACS as Racs_prono,
    da.DIM as FTE_prono,
    da.PSTAFF as Pstaf_prono
    from dimensionamiento_ppp dp
    left join dim_aaron da on dp.Fecha = da.FECHA and da.INTERVALO = dp.INTERVALO and da.Skill = dp.Skill
    WHERE dp.Fecha = '{fecha}' and dp.Intervalo  = '{intervalo}' and dp.skill = {skill};"""
    cursor.execute(query)
    info = cursor.fetchall()
    
    # Cerrar conexion base de datos
    if cursor:
        cursor.close()
        cnx.close()
    
    if info:
        info = info[0]
    
        
    return info
print("Fecha", fecha_dim)
print("Intervalo: ", intervalo_dim)




Fecha 2025-04-17
Intervalo:  08:00


In [11]:
### Funcio hecha con pyautogui. Sin usar
def Whatsapp (grupoWSP):
    # abrir whatsapp
    time.sleep(1)
    pyautogui.press("win")
    pyautogui.write("Whatsapp")
    pyautogui.press("enter")
    pyautogui.press("enter")

    # buscar grupo
    time.sleep(2)
    pyautogui.write(grupoWSP)
    time.sleep(1)
    
    pyautogui.press("down")
    time.sleep(0.5)
    pyautogui.press("enter")
    time.sleep(0.5)
    # Pegar mensaje
    pyautogui.hotkey("ctrl", "v")
    time.sleep(2)
    pyautogui.press("enter")

    # Cerrar app
    time.sleep(2)
    pyautogui.hotkey("alt", "f4")

In [12]:
def segundos_a_hms(segundos):
    minutos = (segundos % 3600) // 60
    segundos = segundos % 60
    return f"{minutos:02}:{segundos:02}"

In [13]:
def evaluear_decisiones(campaña, nds, llamadas_espera, reps_auxiliares, TMO, TMO_dim, staff, reps_avail, ocupacion, oldest):
    """
    Evalúa el estado del call center y genera alertas basadas en métricas clave.

    Condiciones generales:
    - Si el nivel de servicio (nds) es 100% y la llamada más antigua (oldest) es menor a 30s, permite auxiliares.
    - Si el nivel de servicio baja y la ocupación es alta, recomienda suspender auxiliares.
    - Si el TMO aumenta significativamente, genera una alerta.
    """

    acciones = None
    oldest_a_minutos = segundos_a_hms(oldest)
    reps_auxiliares = int(reps_auxiliares)
    reps_avail = int(reps_avail)
    llamadas_espera = int(llamadas_espera)

    # Sacamos el porcentaje de la cantidad de asesores que estan en auxiliar
    porAux = (reps_auxiliares / staff) * 100
    porAux = round(porAux, 2) # Redondeamos a 2 decimales

    # Agentes disponibles
    a_disponibles = math.ceil(reps_avail / 2)

    # Mensajes suspender auxiliares
    msj_title_susp_aux = f"Alerta: Suspender auxiliares"
    msj_susp_aux =  (f"> Nivel de servicio: {nds}%\n> Ocupación: {ocupacion}%\n"
            f"> Llamadas en espera: {llamadas_espera}\n> Llamada mas antigua: {oldest_a_minutos}\n"
            f"> Agentes en auxiliar: {reps_auxiliares} - {staff} ({porAux}% del staff en AUX)")
    
    # Mensajes disponibilidad no programados
    msj_title_no_programados = f"Alerta: Disponibilidad para auxiliares no programados"
    msj_no_programados = f"> Cantidad de agentes disponibles: {a_disponibles}"

    msjTMO = f"Alerta: Incremento en TMO\n> TMO actual: {TMO} (Planificado: {TMO_dim})"

### -------------- CONDICIONALES ------------------ ###

    # Caso 1: Nivel de servicio al 100%
    if nds >= 100:

        # Caso 1,1: Verificamos si el tiempo de espeera de la llamada mas vieja es menor a 30s
        if oldest <= 30:
            if staff >= 5:
                if reps_avail >= (staff * 0.15):
                    acciones = f"\n\n{campaña}\n{msj_title_no_programados}\n{msj_no_programados}"

        # Caso 1,2: Verificamos si el TMO > al TMO Dimensionado
        elif TMO > (TMO_dim * 1.20):  # TMO mayor al 120% del planificado
            acciones = f"\n\n{campaña}\n{msjTMO}"
    
    # Caso 2: Nivel de servicio menor a 100% con ocupación al 100%
    elif ocupacion >= 100:
        # Caso 2,1: Verificamos si el tiempo de espera de la llamada mas vieja es > 0  and < 60
        if 0 < oldest <= 60:

            #Caso 2,1,1: Verificamos que los reps en auxiliar < al 20% de la CO
            if reps_auxiliares >= (staff * 0.20):
                acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}"

                # Caso 2,1,1,1 Verificamos si cumple TMO
                if TMO > (TMO_dim * 1.20):
                    acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}\n{msjTMO}"
            # Caso 2,1,2: Verificamos que se cumpla el TMO
            elif TMO > (TMO_dim * 1.20):
                acciones = f"\n\n{campaña}\n{msjTMO}"
        # Caso 2,2: Si hay llamadas con tiempo de espera mayor a 60s, Verificamos que los auxiliares 
        elif reps_auxiliares >= (staff * 0.20):
            # Caso 2,2,1: En caso de que la cantidad de reps en auxiliar sea mayor al 0.20 del staff. Corroboramos TMO
            if TMO > (TMO_dim * 1.20):
                acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}\n{msjTMO}"
            else:
                acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}"
        # # Caso 2, 3: Si el oldes > 60 y los auxiliares son < a 0.20 del staff. Verificar TMO
        elif TMO > (TMO_dim * 1.20):
            acciones = f"\n\n{campaña}\n{msjTMO}"
    
    # Caso 3: Nivel de servicio menor a 100% con ocupación menor a 100%
    else:
        # Caso 3,1: Verificamos si hay llamadas en espera con tiempo mayor a 30s
        if oldest > 30:
            acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}"

            # Caso 3,1,1 Verificamos el TMO por si hay exceso de aux y exceso en TMO
            if TMO > (TMO_dim * 1.20):
                acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}\n{msjTMO}"
        # Caso 3,2: Si el tiempo de espera de las llamadas es menor a 30_ Verificar uso de auxiliares
        elif reps_auxiliares >= (staff * 0.30) and llamadas_espera > 0:
            acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}"

            # Caso3,2,1: Si superan el 30% Verificamos el TMO
            if TMO > (TMO_dim * 1.20):
                acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}\n{msjTMO}" 

        # Caso 3,3: Solo en el caso de que hayan llamadas en espera y el TMO elevado. Alertamos TMO
        elif llamadas_espera > 0 and TMO > (TMO_dim * 1.20):
            acciones = f"\n\n{campaña}\n{msj_title_susp_aux}\n{msj_susp_aux}"

    return acciones

In [14]:
def envio_wsp(argumento, grupoWhatsapp):
    options = webdriver.ChromeOptions()
    options.add_argument(r"user-data-dir=C:\Users\rocio.nuñez\AppData\Local\Google\Chrome\User Data") 

 
    driver =  webdriver.Chrome("C:\chromedriver.exe", chrome_options= options)
    driver.get("https://web.whatsapp.com/")

    # WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="side"]/div[1]/div/div[2]/div[2]/div/div[1]/p'))).send_keys(grupoWSP)
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="side"]/div[1]/div/div[2]/div/div/div[1]'))).send_keys(grupoWhatsapp)
    WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="side"]/div[1]/div/div[2]/div/div/div[1]'))).send_keys(Keys.ENTER)
#//*[@id="side"]/div[1]/div/div[2]/div/div/div[1]
    time.sleep(1)
    # Copiar el mensaje al portapapeles
    pyperclip.copy(argumento)
    message_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/footer/div[1]/div/span/div/div[2]/div[1]/div/div[1]/p')))
            
                                                                                            #//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[1]/p
    message_box.click()
    time.sleep(2)
    message_box.send_keys(Keys.CONTROL + "v")
    time.sleep(1)
    message_box.send_keys(Keys.ENTER)
    
    time.sleep(5)


    driver.quit()
   


<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\rocio.nuñez\AppData\Local\Temp\ipykernel_4464\3349524281.py:6: SyntaxWarning: invalid escape sequence '\c'
  driver =  webdriver.Chrome("C:\chromedriver.exe", chrome_options= options)


In [ ]:
time.sleep(5)

contador = 0
contadorErrores = 0
contador_NDS = 2
while True:

    try: 
        fecha = datetime.today().strftime("%d/%m/%y")

        if contador == 0:
            # Llamada a la función con el tiempo inicial de 5 segundos solo si el contador es igual a 0 
            iniciar_temporizador(5)

        contador += 1
        contador_NDS += 1

        # ---------------------------- Scrapeo NDS --------------------------------
        if Condicional_NDS == 1 and contador_NDS == 3:  
            df_nds = scrapeoNDS("860;861")
            skills = ["860", "861"]
            for sk in skills:
                for x in df_nds.index:
                    if type(df_nds.loc[x, sk]) == str:
                        df_nds.loc[x, sk] = df_nds.loc[x, sk].replace(",", ".")




            # Analizar primero si el tipo de dato es str, sino no usar relplace
            llamadas_espera_860 = float(df_nds.loc[0, "860"])
            llamadas_espera_861 = float(df_nds.loc[0, "861"])
            # print(llamadas_espera_860)

            nivel_de_servicio_860 = float(df_nds.loc[4, "860"])
            nivel_de_servicio_861 = float(df_nds.loc[4, "861"])
            # print(nivel_de_servicio_860)
            
            agentes_auxiliar_860 = float(df_nds.loc[11, "860"])
            agentes_auxiliar_861 = float(df_nds.loc[11, "861"])
            # print(agentes_auxiliar_860)

            llamadas_abandonadas_860 = float(df_nds.loc[16, "860"])
            llamadas_abandonadas_861 = float(df_nds.loc[16, "861"])

            TMO_860 = float(df_nds.loc[17, "860"])
            TMO_861 = float(df_nds.loc[17, "861"])
            # print("Tmo", TMO_860)

            staff_860 = int(df_nds.loc[9, "860"])
            staff_861 = int(df_nds.loc[9, "861"])
            # print("staff", staff_860)

            s_avail_860 = float(df_nds.loc[10, "860"])
            s_avail_861 =float(df_nds.loc[10, "861"])
            # print("avail", s_avail_860)

            ocupacion_860 = float(df_nds.loc[7, "860"])
            ocupacion_861 = float(df_nds.loc[7, "861"])

            oldest_860 = int(df_nds.loc[2, "860"])
            oldest_861 = int(df_nds.loc[2, "861"])
            # print("ocupacion_860", ocupacion_860)

            # Consumimos la informacion de dimensionamiento_ppp
            dim_860 = consulta_dimensionamiento_ppp(fecha_dim, intervalo_dim, "860")
            dim_861 = consulta_dimensionamiento_ppp(fecha_dim, intervalo_dim, "861")

            Identificador = "*Sistema de alertas automaticas:*"

            # Evaluar Condiciones
            accion_860 = evaluear_decisiones("[COMERCIAL]",nivel_de_servicio_860, llamadas_espera_860, agentes_auxiliar_860, TMO_860, dim_860[4],  staff_860,s_avail_860, ocupacion_860, oldest_860)
            accion_861 = evaluear_decisiones("[CROSSELLING]",nivel_de_servicio_861, llamadas_espera_861, agentes_auxiliar_861, TMO_861, dim_861[4], staff_861, s_avail_861, ocupacion_861,oldest_861)
            # print("Accion: ",accion_860, accion_861)

            if accion_860 is not None and accion_861 is not None:
                mensaje_completo_decisiones = f"{accion_860} \n {accion_861}"
                envio_wsp(mensaje_completo_decisiones, grupoWSP)
            elif accion_861 is not None:
                envio_wsp(accion_861, grupoWSP)
            elif accion_860 is not None:
                envio_wsp(accion_860, grupoWSP)
            contador_NDS = 0

    
        ejecutarReporte("conexiones.acsup")  #Ejecuto el reporte de conexiones
        
        navegarCalcularConcatenarReporteConexiones()  #Consulto y concateno los resultados de ambos skill en df_conexiones

        
        ejecutarReporte("planaTiemposReal.acsup")  #Ejecuto el reporte de tiempo real

        navegarCalcularConcatenarPlanaTiempoReal()  #Consulto y concateno resultados para ambos skill en df_tiempos


        # Ahora voy a mergear ambos dataframes en uno solo llamado 
        df = pd.merge(df_tiempos, df_conexiones[['ID', 'tiempoTranscurrido_min', 'proximoRefrigerio_min']], on='ID', how='left')

        

        # Crear la nueva columna 'Diferncia_refrigerio'
        df['Diferncia_refrigerio'] = df.apply(
            lambda row: row['proximoRefrigerio_min'] - row['Tiempo'] 
            if row['Motivo_AUX'] == 'Refrigerio' else None, 
            axis=1
        )

        
        
        #Creamos la columna para discretizar la hora en intervalos de media hora

        # Si la columna Hora no esta en formato hora la convertimo con la siguiente linea de codigo :
        df['Hora'] = pd.to_datetime(df['Hora'], format='%H:%M').dt.time
        # Aplicamos la función a cada valor en la columna 'Hora' y creamos la nueva columna 'Intervalo'
        df['Intervalo'] = df['Hora'].apply(get_intervalo)

        # Si la columna 'Hora' aún no estaba en texto, puedes convertirla de nuevo a texto:
        df['Hora'] = df['Hora'].astype(str)

        df.to_excel("df1.xlsx", index=False)

        #-------------------------- FILTRADO DE DATOS -----------------------------

        condicion1 = (df['Motivo_AUX'] == 'Baño') & (df['Tiempo'] > 660) # 11 minutos
        condicion2 = (df['Motivo_AUX'] == 'Refrigerio') & (df['Diferncia_refrigerio'] < -60) # Cuando la diferencia es menor a -60
        condicion3 = (df['Motivo_AUX'] == 'GestOut/Res Reclamos') &  (df['Tiempo'] > 1200) # 20 minutos
        condicion4 = (df['Motivo_AUX'] == 'Problemas Sistemas') & (df['Tiempo'] > 120) # 2 minutos
        condicion5 = (df['Motivo_AUX'] == 'HOLD') & (df['Tiempo'] > 360) # 6 minutos    
        
    


        # Inicializar condicion6 como una serie de booleanos con valor False
        condicion6 = pd.Series([False] * len(df), index=df.index)

        # Calcular el menor tiempo para 'Momento T'
        menorTiempo = df[df['Motivo_AUX'] == 'REIMAGINA  MOMENTO T']['Tiempo'].min()
        umbral = 1800

        
        # Verificar si el menor tiempo de 'Momento T' supera los 1800 segundos
        if menorTiempo > umbral:
            # Condición para 'Momento T' solo si el menor tiempo supera los 1800 segundos
            condicion6 = (df['Motivo_AUX'] == 'REIMAGINA  MOMENTO T') & (df['Tiempo'] > umbral)

                                            

        #Si el df trae info se filtra 
        if not df.empty:

            # Aplicar el filtro combinando las condiciones con el operador OR
            df = df[condicion1  | condicion2 | condicion3 | condicion4 | condicion5  | condicion6 ]



    
        
        # Enviar mensaje si el df_tiene información 
        if not df.empty:
            
            df.to_excel("Alertas.xlsx", index=False)
            if publicar == 1:
                    
                    # Eliminar columnas
                    df = df.drop(['tiempoTranscurrido_min', 'proximoRefrigerio_min', 'Diferncia_refrigerio'], axis=1)

                
                    #Iniciar Conexión
                    cnx = iniciarconexion('localhost','db_gtr','root','sqlPassGTR2024*')

                    data = [tuple(row) for row in df.values]

                    columns = df.columns.tolist()
                
                    columns_str = ', '.join(columns)
                    placeholders = ', '.join(['%s' for _ in columns])
                    sql = f"INSERT INTO {dbTableName} ({columns_str}) VALUES ({placeholders})"
                
                    # Ejecutar la sentencia SQL INSERT
                    cursor = cnx.cursor()
                    cursor.executemany(sql, data)

                    cnx.commit()
                    cnx.close()

            
                


            #-------------------------- ENVIO DE DATOS POR WSP -----------------------------

            # Antes que nada convierto el tiempo en minutos
            df["Tiempo"] = df["Tiempo"] / 60
            df["Tiempo"] = df["Tiempo"].astype(float)
            df["Tiempo"] = df["Tiempo"].round(1)

            #Generar el mensaje
            
            mensaje_inicial = "Sistema de alertas automáticas: \n \n"
            mensajes = [f'[ _{row['Nombre']}_ ] *"{row['Motivo_AUX']}"* con: {row['Tiempo']} minutos.' for index, row in df.iterrows()]
            mensaje_completo = mensaje_inicial + "\n \n".join(mensajes)

            # Envio por wsp
            envio_wsp(mensaje_completo, grupoWSP)


        
        #Se limpia el dataframe para utilizarlo en un proximo intervalo
        df_conexiones = df_conexiones.drop(df_conexiones.index)
        df_tiempos = df_tiempos.drop(df_tiempos.index)

        
        interfaceCuentaRegresiva(3)

        
    except ZeroDivisionError as zeroDivision:
        print("Zero Division Error")
        now = datetime.now()
        if now.hour == 7 or now.hour == 8:
            interfaceCuentaRegresiva(5)
            print("⚠️ Se Configuro el tiempo de espera a 5 minutos siendo que falta poco para la jornada laboral ⚠️")
        else:
            print("Se configuro la interfaz para un tiempo de espera de  30 minutos ya que el comienzo de la jornada laboral esta \
                  lejos de comenzar")
            interfaceCuentaRegresiva(30)
    except IndexError:
        # Este error aparece normalmente al comienzo o al final de la jornada, cuando no hay datos para exportar
        print(f"Index error {IndexError}")
        if now.hour == 7 or now.hour == 8:
            interfaceCuentaRegresiva(5)
            print("⚠️ Se Configuro el tiempo de espera a 5 minutos siendo que falta poco para la jornada laboral ⚠️")
        else:
            print("Se configuro la interfaz para un tiempo de espera de  30 minutos ya que el comienzo de la jornada laboral esta lejos de comenzar")
            
            interfaceCuentaRegresiva(30)
        
        
    except Exception as error:
        print("Error de sistema: ",
              traceback.print_exc(),
              error)
        contadorErrores += 1
        if contadorErrores == 3:
            sys.exit(f"Se finalizo el bucle por el error {error}")
            break
        mensajeDeErrorCompleto = f"*Sistema de alertas automaticas*\nError de sistema:\nTipo  de error: {type(error)}\nCantidad de errores: {contadorErrores}\nError: {error}"
        contador_NDS = 2
        envio_wsp(mensajeDeErrorCompleto, contactoError)
        ### Funcion que busca todos los procesos que se estan corriendo en la pc y cierra los relacionados \ 
        # a los reportes de avaya
        nombre_buscado = "Estado de la cola de espera/agente superior"
        nombre_buscado2 = "Report Engine"
        def enum_handler(hwnd, extra):
            if win32gui.IsWindowVisible(hwnd):
                titulo = win32gui.GetWindowText(hwnd)
                if titulo and nombre_buscado.lower() in titulo.lower() or titulo and nombre_buscado2.lower() in titulo.lower():
                    print(f"Ventana encontrada: {titulo}")
                    try:
                        _, pid = win32process.GetWindowThreadProcessId(hwnd)
                        proc = psutil.Process(pid)
                        print(f" ✅ Cerrando proceso: {proc.name()} (PID: {pid})")
                        proc.terminate()
                    except Exception as e:
                        print(f" ❌ Error al cerrar el proceso: {e}")

        win32gui.EnumWindows(enum_handler, None)
        # Una vez cerrados todos los reportes esperamos 1 minuto y comenzamos de nuevo el bucle
        interfaceCuentaRegresiva(1)



C:\Users\rocio.nuñez\AppData\Local\Temp\ipykernel_4464\2856317680.py:251: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'HOLD' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Estado'] == 'OTHER', 'Motivo_AUX'] = 'HOLD'
C:\Users\rocio.nuñez\AppData\Local\Temp\ipykernel_4464\2856317680.py:251: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'HOLD' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['Estado'] == 'OTHER', 'Motivo_AUX'] = 'HOLD'
C:\Users\rocio.nuñez\AppData\Local\Temp\ipykernel_4464\2856317680.py:251: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'HOLD' has dtype incompatible with float64, please explicitly cast to a compatible dtype

Error de sistema:  None invalid command name ".!ctklabel2.!label"
